# Build an Agent

LangChain supports the creation of [agents](/docs/concepts/agents), or systems that use [LLMs](/docs/concepts/chat_models) as reasoning engines to determine which actions to take and the inputs necessary to perform the action.
After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via [tool-calling](/docs/concepts/tool_calling).

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## End-to-end agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!

In [2]:
!pip install langchain_tavily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 123.7 MB/s eta 0:00:00


In [4]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 11.0 MB/s eta 0:00:00


In [6]:
!pip install -U langchain-anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 8.8 MB/s eta 0:00:00


In [8]:
import os
os.environ["TAVILY_API_KEY"] = ""

In [12]:
!pip install -U langchain-mistralai

In [13]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model("mistralai:mistral-small-latest")
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [17]:
import os
os.environ["MISTRAL_API_KEY"] = ""
os.environ["MISTRAL_API_KEY"] = ""

In [18]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I life in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I life in SF.


HTTPStatusError: Error response 401 while fetching https://api.mistral.ai/v1/chat/completions: {"detail":"Unauthorized"}

In [ ]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================

[{'text': 'Let me search for current weather information in San Francisco.', 'type': 'text'}, {'id': 'toolu_011kSdheoJp8THURoLmeLtZo', 'input': {'query': 'current weather San Francisco CA'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_011kSdheoJp8THURoLmeLtZo)
 Call ID: toolu_011kSdheoJp8THURoLmeLtZo
  Args:
    query: current weather San Francisco CA
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather San Francisco CA", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, CA", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America',

In [ ]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

For more details, see our [Installation guide](/docs/how_to/installation).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

### Tavily

We will be using [Tavily](/docs/integrations/tools/tavily_search) (a search engine) as a tool.
In order to use it, you will need to get and set an API key:

```bash
export TAVILY_API_KEY="..."
```

Or, if in a notebook, you can set it with:

```python
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()
```

In [19]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751115322, 'localtime': '2025-06-28 05:55'}, 'current': {'last_updated_epoch': 1751114700, 'last_updated': '2025-06-28 05:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 233, 'wind_dir': 'SW', 'pressure_mb': 1014.0, 'pressure_in': 29.93, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 25, 'feelslike_c': 13.2, 'feelslike_f': 55.8, 'windchill_c': 9.9, 'windchill_f': 49.8, 'heatindex_c': 10.6, 'heatindex_f': 51.2, 'dewpoint_c': 9.8, 'dewpoint_f': 

In [ ]:
# | output: false
# | echo: false

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-latest")

In [20]:
pip install -qU "langchain[mistralai]"

In [21]:
import getpass
import os

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("mistral-large-latest", model_provider="mistralai")

In [22]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

"Hello! How can I help you today? Let's have a friendly conversation. 😊 How are you doing?"

In [23]:
model_with_tools = model.bind_tools(tools)

In [24]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I assist you today?

Tool calls: []


In [25]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in SF'}, 'id': 'MstiMJPQN', 'type': 'tool_call'}]


In [26]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [27]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I assist you today?


In [28]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (q5I7mjujI)
 Call ID: q5I7mjujI
  Args:
    query: weather in SF
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751115535, 'localtime': '2025-06-28 05:58'}, 'current': {'last_updated_epoch': 1751114700, 'last_updated': '2025-06-28 05:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/

We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.

In [29]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (OLUcQk0Ph)
 Call ID: OLUcQk0Ph
  Args:
    query: weather in SF
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751115535, 'localtime': '2025-06-28 05:58'}, 'current': {'last_updated_epoch': 1751114700, 'last_updated': '2025-06-28 05:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/

In [30]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather in San Francisco| is mist|y with a temperature| of 12|.8°C| (|55.0|°F).| The wind is blowing| at a speed of| 2.5| mph from| the southwest,| and| the hum|idity is 9|3%. The| visibility| is 16|.0 km| (|9.0| miles), and the| UV| index is 0|.0|. The feels|-|like temperature is |13.2|°C (5|5.8°|F)| |.
|
For| the forecast|, on| Saturday|, June 2|8, 2|025,| the| day|time| temperature is expected to| be 6|4°F,| and the nighttime| temperature is| expected| to be 5|4|°F,| with no| precipitation and| a UV| index of 1|1 |.|

In [31]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [32]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [33]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello Bob! How can I assist you today?


In [34]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob! How can I help you today?


Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If you want to start a new conversation, all you have to do is change the `thread_id` used

In [35]:
# highlight-next-line
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I don't have access to personal information about you, including your name. If you'd like to share it or need assistance with something specific, feel free to let me know!
